In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
from IPython.display import Image
import random

import warnings
warnings.filterwarnings("ignore")

In [20]:
data=pd.read_pickle('df_final.pkl')
#data.drop('weights',inplace=True,axis=1)
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,...,sold_lag_35,sold_lag_42,sold_lag_49,sold_lag_56,item_sold_avg,avg_sold_storewis,avg_sold_statewis,avg_sold_catwis,avg_sold_store_item,avg_sold_state_item
45735000,14370,1437,3,1,0,0,1501,0,11506,3,...,0.0,1.0,1.0,2.0,0.216553,1.304688,1.21582,0.561035,0.321533,0.335938
45735001,14380,1438,3,1,0,0,1501,1,11506,3,...,0.0,0.0,0.0,2.0,0.259766,1.304688,1.21582,0.561035,0.253906,0.204346
45735002,14390,1439,3,1,0,0,1501,0,11506,3,...,0.0,1.0,0.0,1.0,0.076660,1.304688,1.21582,0.561035,0.156982,0.098938
45735003,14400,1440,3,1,0,0,1501,1,11506,3,...,0.0,3.0,8.0,2.0,2.011719,1.304688,1.21582,0.561035,1.694336,2.935547
45735004,14410,1441,3,1,0,0,1501,1,11506,3,...,1.0,0.0,1.0,0.0,0.755371,1.304688,1.21582,0.561035,0.958984,0.968750


# splitting

In [22]:
def split_data(data):

    #Product details upto day 1913 is used as training dataset
    X_train = data[data['d']<=1913]
    y_train = X_train['sales']
    X_train.drop('sales',axis = 1,inplace = True)
    
    # Product details from day 1914 to 1941 is used as validation dataset
    X_val = data[(data['d']>=1914) & (data['d']<=1941)]
    y_val = X_val['sales']
    X_val.drop(['sales'],axis = 1,inplace = True)
    
    # Product details starting from day 1942 is used as testing dataset
    X_test = data[(data['d']>=1942)]
    X_test.drop(['sales'],axis = 1,inplace = True)
    
    #print("Train dataset",X_train.shape,y_train.shape)
    #print("Validation dataset",X_val.shape,y_val.shape)
    #print("Test dataset",X_test.shape)
    return(X_train,y_train,X_val,y_val,X_test)

In [23]:
def reshape(input):
    value = np.reshape(input, (-1, 28),order = 'F')
    return value

In [24]:
from tqdm import tqdm

In [25]:
def submission_term(val_predict,test_predict):
        
    val_pred =reshape(val_predict)
    test_pred=reshape(test_predict)
    
    submission=pd.read_csv("sample_submission.csv")
    
    for ind in tqdm(range(28)):
        submission.iloc[:30490,ind+1]=val_pred[:,ind]
        submission.iloc[30490:,ind+1]=test_pred[:,ind]
        
    return submission

In [29]:
xtrain,ytrain,xval,yval,xtest=split_data(data)


# Linear Regression

In [14]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

In [16]:
model = LinearRegression(n_jobs=-1)
model.fit(xtrain,ytrain)

LinearRegression(n_jobs=-1)

In [9]:
yval_pred  = model.predict(xval)
ytest_pred = model.predict(xtest)

In [10]:
rmse = mse(yval,yval_pred)

print(rmse)

submission1 = submission_term(yval_pred,ytest_pred)
#submission1.to_csv('sample_submission_new.csv')
#submission1.head()

5.020013


100%|██████████████████████████████████████████| 28/28 [00:00<00:00, 621.98it/s]


# Ridge Regression

In [16]:
from sklearn import linear_model

In [17]:
model2=linear_model.Ridge()
model2.fit(xtrain,ytrain)

NameError: name 'xtrain' is not defined

In [13]:
yval_pred  = model2.predict(xval)
ytest_pred = model2.predict(xtest)

In [14]:
rmse = mse(yval,yval_pred)

print(rmse)
submission2=submission_term(yval_pred,ytest_pred)
submission2.to_csv("sample_submission_new.csv")
submission2.head()

5.025259


100%|█████████████████████████████████████████| 28/28 [00:00<00:00, 1206.58it/s]


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.494873,0.671143,0.594971,0.410156,0.790527,0.484375,0.981445,0.449951,1.213379,...,0.741943,1.250977,1.158447,0.719482,0.781982,0.730225,1.053711,0.481689,1.223389,0.851074
1,HOBBIES_1_002_CA_1_validation,0.452881,0.240234,0.249023,0.427734,0.289795,0.473145,0.614014,0.446533,0.314209,...,0.345215,0.532715,0.575684,0.253662,0.192871,0.353271,0.287354,0.370361,0.473145,0.481689
2,HOBBIES_1_003_CA_1_validation,0.259766,0.263184,0.133301,0.141846,0.316162,0.895020,0.695801,0.307129,0.501709,...,0.759033,0.792480,0.985107,0.369385,0.452637,0.484619,0.506836,0.677002,0.681885,0.854980
3,HOBBIES_1_004_CA_1_validation,2.083984,1.457275,1.212891,1.629639,1.593750,2.004150,2.822998,1.883789,1.739990,...,2.035400,2.268066,3.036865,2.040527,1.673828,1.769775,1.499268,1.791992,3.079590,2.970215
4,HOBBIES_1_005_CA_1_validation,1.173828,1.068848,1.486084,1.108154,1.218262,1.354980,1.568115,1.189697,1.430908,...,1.203369,1.327393,1.647461,0.996338,1.036133,1.124023,1.348877,1.110596,1.401367,1.921631


## XG Boost Regression

In [32]:
import xgboost as xg
from xgboost import XGBRegressor

xg = XGBRegressor()
xg.fit(xtrain,ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [33]:
yval_pred  = xg.predict(xval)
ytest_pred = xg.predict(xtest)

In [34]:
rmse = mse(yval,yval_pred)

print(rmse)
submission2=submission_term(yval_pred,ytest_pred)
submission2.to_csv("sample_submission_new.csv")
submission2.head()

4.823915


100%|██████████████████████████████████████████| 28/28 [00:00<00:00, 697.87it/s]


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.548948,0.635639,0.565226,0.448413,0.888147,0.572936,0.908161,0.466230,0.958791,...,0.504472,0.934570,0.897020,0.603620,0.700753,0.674214,0.731406,0.533479,1.164374,0.790104
1,HOBBIES_1_002_CA_1_validation,0.477269,0.307712,0.307712,0.316907,0.279971,0.441175,0.457917,0.309689,0.228686,...,0.277185,0.361596,0.358310,0.270622,0.209709,0.333592,0.225638,0.353694,0.336816,0.336816
2,HOBBIES_1_003_CA_1_validation,0.289248,0.281285,0.155231,0.153855,0.308794,0.594037,0.515963,0.242058,0.349913,...,0.469556,0.675390,0.854530,0.355807,0.466470,0.376630,0.314255,0.451043,0.562536,0.588482
3,HOBBIES_1_004_CA_1_validation,2.247004,1.579988,1.373273,1.577932,2.078691,2.256279,3.103484,1.917374,1.569374,...,2.290147,2.528043,3.443142,1.848240,1.633349,2.022755,1.504826,1.898533,3.336737,2.783947
4,HOBBIES_1_005_CA_1_validation,1.126581,1.031596,1.111398,0.978862,1.266147,1.487973,1.510789,1.101612,1.088830,...,1.015085,1.261492,1.585064,0.933491,1.001521,0.782883,1.080777,0.978209,1.299791,1.855562


## KNN

In [35]:
from sklearn import neighbors
knn = neighbors.KNeighborsRegressor()
knn.fit(xtrain,ytrain)

KNeighborsRegressor()

In [39]:
yval_pred  = knn.predict(xval)
ytest_pred = knn.predict(xtest)

In [40]:
rmse = mse(yval,yval_pred)

print(rmse)
submission2=submission_term(yval_pred,ytest_pred)
submission2.to_csv("sample_submission_new.csv")
submission2.head()

5.723131


# lgbm

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
for i in range(15):
    lr = np.round(np.random.rand()/10,3)
    num_leaves = np.random.randint(30,150)
    min_data_in_leaf = np.random.randint(50,150)

    lgb = LGBMRegressor( learning_rate=lr ,
                            num_leaves=num_leaves ,
                            min_data_in_leaf=min_data_in_leaf)

    lgb.fit(xtrain, ytrain)

    # After training the model, we are trying to predict the model on X_val to check it's accuracy
    y_pred = lgb.predict(xval)

    # Now that it has predicted the values for X_val, we are calculating it's rmse
    rmse = mse(yval,y_pred)
    
    print(f"For learning rate {lr}, num_leaves {num_leaves} and min_data_in_leaf {min_data_in_leaf} the RMSE is {rmse} ")
    print('*'*80)

In [ ]:
#best parameter:
min_data_in_leaf=114
lr =0.089
num_leaves= 146 

best_lgb = LGBMRegressor( learning_rate=lr ,num_leaves=num_leaves ,min_data_in_leaf=min_data_in_leaf)
best_lgb.fit(xtrain,ytrain)

yval_pred  = best_lgb.predict(xval)
ytest_pred = best_lgb.predict(xtest)

rmse = mse(yval,yval_pred)
print(rmse)

In [ ]:
submission3=submission_term(yval_pred,ytest_pred)
submission3.to_csv("best_sample_submission.csv")
submission3.head()

In [ ]:
import lightgbm as lgb
lgb.plot_importance(best_lgb, figsize=(10, 12))

## LSTM


In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

In [5]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)



KeyboardInterrupt: 

In [7]:
data.to_pickle("df_final_4.pkl",protocol=4)

In [18]:
print('x_train shape:', xtrain.shape)
print('x_test shape:', xtest.shape)

x_train shape: (12592370, 33)
x_test shape: (853720, 33)


In [ ]:
model = Sequential()
#model.add(Embedding(max_features, 128))
model.add(LSTM((1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_absolute_error',
              optimizer='adam',
              metrics=['accuracy'])


model.fit(xtrain, ytrain,
          batch_size=32,
          epochs=5,
          validation_data=(xval, yval))

yval_pred  = model.predict(xval)
ytest_pred = model.predict(xtest)


rmse = mse(yval,yval_pred)
print(rmse)